In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import time 
import requests
import json
np.random.seed(42)

In [2]:
url = 'https://api.pushshift.io/reddit/search/submission/?subreddit=harrypotter&size=500'

In [8]:
url_next_page = 'https://api.pushshift.io/reddit/search/submission/?subreddit=harrypotter&size=500&after=30d'

In [3]:
def test_page(url):
    try:
        res = requests.get(url,headers = {'User-agent':'OAUTH 2'})
    except:
        return('not valid URL')
    try:
        res.status_code < 400
        return(res.json())  
    except:
        return('status code error',res.status_code)

In [5]:
data = test_page(url)

In [7]:
type(data)

dict

In [18]:
def get_page(data):
    main = data['data']
    for i in range(1,len(main)):
                temp_dict = {}
                temp_dict['posts'] = main[i]['title']
                posts.append(temp_dict)
    return('Page Scraped...' + url)

In [19]:
posts = []


In [20]:
get_page(data)

'Page Scraped...https://api.pushshift.io/reddit/search/submission/?subreddit=harrypotter&size=500'

In [21]:
len(posts)

499

In [24]:
def next_page(data,url,amount_pages):
    for i in range(amount_pages):
        try:
            url.index('&after=')
        except:
            url = url+'&after='

In [ ]:
['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score', 'is_self']

In [29]:
def query_pushshift(subreddit, kind='submission', skip=30, times=6,sort = False,
                    subfield = ['title', 'selftext', 'subreddit', 'created_utc', 'author', 'num_comments', 'score', 'is_self']):

    stem = "https://api.pushshift.io/reddit/search/{}/?subreddit={}&size=500".format(kind, subreddit)
    mylist = []
    
    for x in range(1, times):
        
        URL = "{}&after={}d".format(stem, skip * x)
        print(URL)
        response = requests.get(URL)
        assert response.status_code == 200
        mine = response.json()['data']
        df = pd.DataFrame.from_dict(mine)
        mylist.append(df)
        time.sleep(2)
        
    full = pd.concat(mylist)
    
    if kind == "submission":
        
        full = full[subfield]
        
        full = full.drop_duplicates()
        
        full = full.loc[full['is_self'] == True]
        
    def get_date(created):
        return dt.date.fromtimestamp(created)
    
    _timestamp = full["created_utc"].apply(get_date)
    
    full['timestamp'] = _timestamp

    print(full.shape)
    
    return full 

In [30]:
df = query_pushshift('harrypotter')

https://api.pushshift.io/reddit/search/submission/?subreddit=harrypotter&size=500&after=30d
https://api.pushshift.io/reddit/search/submission/?subreddit=harrypotter&size=500&after=60d
https://api.pushshift.io/reddit/search/submission/?subreddit=harrypotter&size=500&after=90d
https://api.pushshift.io/reddit/search/submission/?subreddit=harrypotter&size=500&after=120d
https://api.pushshift.io/reddit/search/submission/?subreddit=harrypotter&size=500&after=150d
(1672, 9)


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False



In [31]:
df.head()

,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp
0,"Is there a specific wand you consider ""your"" w...","Mine is redwood (though because in real life, ...",harrypotter,1533574604,Therealtylerwickett,5,1,True,2018-08-06
1,Do any of you guys remember that old animated ...,I used to use a website called the way back ma...,harrypotter,1533574690,Therealtylerwickett,2,1,True,2018-08-06
2,Maybe we are not interpreting correctly the fa...,,harrypotter,1533574822,Otroweasley,7,1,True,2018-08-06
3,Hogsmeade Permission,"in the 6th book, harry has permission to go do...",harrypotter,1533574875,Klandan54,9,1,True,2018-08-06
4,What do you think Ron called Snape in Prisoner...,It was enough to offend Hermione. Perhaps the ...,harrypotter,1533575078,KaiserKCat,18,1,True,2018-08-06


In [34]:
for i in df['selftext'].values:
    print(i)

Mine is redwood (though because in real life, redwood doesn't really look red I'd probably use bloodwood or paduak), dragon heartstring, 13 1/4 inches, flexible. Cause when I was a kid, I'd always try to get it when I was playing that virtual ollivander wand shop game. 
I used to use a website called the way back machine to let me access an older version of the harry potter website to play the flash games, but that hasn't been working for me recently. Do any of you guys have clever tricks up your sleeves? 

in the 6th book, harry has permission to go do hogsmeade. as far as i know it wasn't mentioned that he got permission from the dursleys, so should he not lack permission?
It was enough to offend Hermione. Perhaps the C word?
How do you think the dorm situation was handled when it comes to trans kids at Hogwarts? I know that students aren't allowed to go to the dorm rooms of the opposite sex, and that there is a sex-based charm placed on the girl's stairs preventing boys from going u